In [ ]:
import json
from glob import glob
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')

# rdir = '../results/lpc/results_22-05-13r1/'
rdir = '../results/lpc/results_22-05-14/'

In [ ]:
frames = []
# cols = [
#     'algorithm',
#     'random_state',
#     'time_time',
#        ]
for f in glob(rdir+'/*.json'):
#     print(f)
    with open(f,'r') as file:
        try:
            d = json.load(file)
        except Exception as e:
            print('!!!Failed to load',f)
            print(e)
            continue
    frames.append(d)    
df_results = pd.DataFrame.from_records(frames)
print(len(frames),'records')
print(frames[0].keys())

df_results['ml_name'] = df_results['algorithm'].apply(lambda x: x.split('_pmc')[0].split('_mc')[0])
df_results['postprocessing'] = df_results['algorithm'].apply(lambda x: 'MC' if '_mc' in x else x)
df_results['postprocessing'] = df_results['postprocessing'].apply(lambda x: 'PMC' if '_pmc' in x else x)
df_results['postprocessing'] = df_results['postprocessing'].apply(lambda x: 'none' if not x in ['MC','PMC'] else x)
df_results.loc[df_results['ml_name']=='lr_cv','ml_name'] = 'lr'
# df_results['MC'] = df_results['algorithm'].apply(lambda x: '_mc' in x)
# df_results['PMC'] = df_results['algorithm'].apply(lambda x: '_pmc' in x)
df_results.groupby('algorithm')['random_state'].count()

In [ ]:
df_results['postprocessing'].unique()

In [ ]:

df_results.groupby('algorithm').mean().round(3)

In [ ]:
import pdb
def nice_stat(x):
#     pdb.set_trace()
    return f'{x.mean():.2f}$\pm${x.sem():.2f}'
def nice_stats(x):
    return x.apply(lambda x: nice_stat(x))


In [ ]:
metrics = ['roc_auc','auprc', 'MC_loss', 'PMC_loss', 'DC_loss' ]
test_metrics = [m+'_test' for m in metrics]
train_metrics = [m+'_train' for m in metrics]
# df_results.groupby('algorithm')[].mean().round(3)
df_results.groupby(['ml_name','postprocessing'])[test_metrics].mean().round(3)

# for m in test_metrics:
#     df_results[m+'_nice'] = 
df_results.groupby(['ml_name','rho','n_bins','postprocessing'])[test_metrics].apply(nice_stats)

In [ ]:
order = df_results.groupby('ml_name').groups.keys()


In [ ]:
df_plt = df_results.copy()
# df_plt = df_plt.loc[df_plt.gamma==0.01]
df_plt = df_plt.loc[df_plt.n_bins==10]
df_plt = df_plt.loc[df_plt.ml_name=='lr']
    
    
# for m in test_metrics:
for m in ['MC_loss_test','PMC_loss_test','DC_loss_test']:
#     plt.figure(figsize=(10,10))
    plt.figure()
    g=sns.catplot(
        kind='box',
        data=df_plt,
#         x='ml_name',
        y='gamma',
        x=m,
#         order=order,
        hue='postprocessing',
        hue_order=['None','MC','PMC'],
        row='alpha',
#         col='gamma',
        col='rho',
#         col='n_bins',
        showfliers=False,
        facet_kws=dict(sharey=False),
        sharey=False,
#         join=False,
#         aspect=0.5
    )
#     g.set(suptitle=m)

In [ ]:
df_plt = df_results.copy()
# df_plt = df_plt.loc[df_plt.gamma==0.01]
df_plt = df_plt.loc[df_plt.n_bins==10]
df_plt = df_plt.loc[df_plt.ml_name=='lr']
    
row,col,hue=('alpha','gamma','rho') 
# row,col,hue=('rho','gamma','alpha') 
# for m in test_metrics:
for m in ['roc_auc_test','MC_loss_test','PMC_loss_test','DC_loss_test']:
#     plt.figure(figsize=(10,10))
#     for row,col,hue in [('alpha','gamma','rho'),('alpha','rho'),('rho','gamma')]:
    plt.figure()
    g=sns.catplot(
        kind='box',
#         dodge=True,
        data=df_plt,
#             x='ml_name',
        x='postprocessing',
        y=m,
        order=['none','MC','PMC'],
        hue=hue,
#         hue_order=['None','MC','PMC'],
        row=row,
#         col='gamma',
        col=col,
#         col='n_bins',
        showfliers=False,
#         notch=True,
#         facet_kws=dict(sharex=False),
        facet_kws=dict(sharey=False),
#         sharex=False,
        sharey=False,
#         join=False,
#         aspect=0.5
    )
    #     g.set(suptitle=m)

In [ ]:
sns.displot(data=df_plt, x="alpha", y="gamma", hue="MC_loss_test", kind="kde")

In [ ]:
df_plt = df_results.copy()
    
# for m in test_metrics:
for m in ['MC_loss_test','PMC_loss_test','DC_loss_test']:
#     plt.figure(figsize=(10,10))
    plt.figure()
    g=sns.catplot(
        kind='box',
        data=df_plt,
        x='ml_name',
        y=m,
        hue='postprocessing',
        hue_order=['None','MC','PMC'],
        row='alpha',
        col='gamma',
        order=order,
        showfliers=False,
        facet_kws=dict(sharey=False),
        sharey=False
#         join=False
    )
#     g.set(suptitle=m)

In [ ]:
df_plt = df_results.copy()
    
for m in test_metrics:
    plt.figure(figsize=(10,10))
    sns.boxplot(
        data=df_plt,
        y='ml_name',
        x=m,
        hue='postprocessing',
#         order=order,
#         join=False
    )